In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/ki-lab-ss23
%ls

/content/drive/MyDrive/ki-lab-ss23
0_Simple_NN.py             LICENSE            roberta-base-sentence-relation/  Task03.ipynb
data/                      my_awesome_model/  Task00.ipynb                     Task04.ipynb
final-project/             README.md          Task01.ipynb                     Task05.ipynb
GPU_Lab_Remote_Access.pdf  requirements.txt   Task02.ipynb                     Task06.ipynb


In [3]:
import sys
import os
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
print(dir1)

/content/drive/MyDrive


In [4]:
! pip install -U accelerate
! pip install -U transformers

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import plot_tree

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased", truncation=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
raw_data      = pd.read_csv('data/sentence-relations/train.csv', index_col='id')
raw_submissions = pd.read_csv('data/sentence-relations/test.csv', index_col='id')

# Remove chinese
raw_data = raw_data[raw_data['lang_abv'] != 'zh']

training_data, test_data = train_test_split(raw_data, test_size=0.2, random_state=42)

training_data.head()

,premise,hypothesis,lang_abv,language,label
id,,,,,
feca7a61d7,What idiots girls are!,Girls can put emotions ahead of logical though...,en,English,1
75cbac107c,"Hey, no problem, a fine policy.","No worries, a good policy.",en,English,0
a98caa4b17,He reverted to his former point of view.,He went back to his previous thoughts about vi...,en,English,1
7b0e870532,She's smiling but her eyes are closed.,Her eyes would not open but she is smiling.,en,English,1
f803a85af1,Denk dann über die Rolle von Gesetzen und Vert...,Gesetze und die Wirtschaft are nicht verbunden.,de,German,2


In [7]:
bert_model_name = 'distilbert/distilbert-base-uncased'
max_len         = 128
tokenizer       = AutoTokenizer.from_pretrained(bert_model_name)

In [9]:
from torch.utils.data import Dataset, DataLoader

class SentenceRelationTransformerDataset(Dataset):

    def __init__(self, data, tokenizer):
        self.data               = data
        self.sentence_relations = []
        self.labels             = []
        self.tokenizer          = tokenizer

        self.perform_preprocessing()

    def __len__(self):
        return len(self.data)

    def get_max_input_length(self):

        max_length = 0

        for sentence_pair in self.sentence_relations:
            max_length = max(max_length, len(sentence_pair[0]), len(sentence_pair[1]))

        return max_length

    def perform_preprocessing(self):
        self.sentence_relations = []
        self.labels             = []

        for i in range(len(self.data)):
            premise    = self.data.iloc[i]['premise']
            hypothesis = self.data.iloc[i]['hypothesis']

            input_string = f"{premise} [SEP] {hypothesis}"

            input_tokenized = self.tokenizer(input_string, padding='max_length', max_length=512, truncation=True)

            label = self.data.iloc[i]['label']

            self.labels.append(label)
            self.sentence_relations.append(input_tokenized)

    def __getitem__(self, idx):
        temp_dict = self.sentence_relations[idx]
        temp_dict['label'] = self.labels[idx]

        return temp_dict


train_dataset = SentenceRelationTransformerDataset(training_data, tokenizer)
test_dataset  = SentenceRelationTransformerDataset(test_data, tokenizer)

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}
num_labels = len(id2label)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=num_labels, id2label=id2label, label2id=label2id
)
model.to('cpu')
print("")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=None,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

IndexError: index out of range in self